In [3]:
import pandas as pd

In [32]:
from zipfile import ZipFile

In [33]:
from  pathlib import Path
import calendar

In [8]:
import datetime

In [4]:
path = Path(r'C:\Users\nyzhur\Git\pg-dt-portal-for-nz\codes\periods.xlsx')

In [5]:
periods = pd.read_excel(path, sheet_name ='period_lbl')

In [8]:
periods.sort_values(by=['period_lbl', 'label_num'])

,period_code,period_lbl,label_num,period_batch
3,4,2MATs: Dec 2023,2MATs: 2023 (12) Dec,0
1,2,2MATs: Jun 2023,2MATs: 2023 (06) Jun,0
0,1,2MATs: Mar 2023,2MATs: 2023 (03) Mar,0
4,5,2MATs: Mar 2024,2MATs: 2024 (03) Mar,0
2,3,2MATs: Sep 2023,2MATs: 2023 (09) Sep,0
...,...,...,...,...
82,83,Month: Oct 2022,Month: 2022 (10) Oct,0
94,95,Month: Oct 2023,Month: 2023 (10) Oct,0
69,70,Month: Sep 2021,Month: 2021 (09) Sep,0
81,82,Month: Sep 2022,Month: 2022 (09) Sep,0


In [9]:
def get_label_num(time_period_type, year, month):
    return f"{time_period_type.split('/')[0]}: {year} ({str(month).zfill(2)}) {calendar.month_abbr[month]}"

In [10]:
year = 2023
month = 3
period = '2MATs/ 104 we'

In [11]:
get_label_num(period, year, month)

'2MATs: 2023 (03) Mar'

In [12]:
f ='2MATs: 2023 (03) Mar'
print(f)
print(get_label_num(period, year, month))

2MATs: 2023 (03) Mar
2MATs: 2023 (03) Mar


In [12]:
periods_1= pd.DataFrame(columns = periods.columns)
periods_1

,period_code,period_lbl,label_num,period_batch


In [13]:
periods_1 = periods_1.iloc[10:,]
periods_1

,period_code,period_lbl,label_num,period_batch


In [ ]:
count = periods_1['period_code'].max()
for lbl, num in periods[['period_lbl','label_num']].values:
        if  not lbl in periods_1['period_lbl'].values:
            print(lbl)
            

            periods_1 = pd.concat([periods_1, row], ignore_index=True)

2MATs: Mar 2023


NameError: name 'row' is not defined

In [154]:
periods_1['period_code'].unique()

array([21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
       38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54,
       55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
       72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88,
       89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119], dtype=object)

In [13]:
blya = '3MMT: Sep 2022'

In [139]:
blya.split(':')[1][-4:]


'2022'

In [57]:
def get_num_from_period(string):
    period, date = string.split(':')
    month = date[1:4]
    year =  date[-4:] 
    month_number = list(calendar.month_abbr).index(month)
    return f'{period}: {year} ({str(month_number).zfill(2)}) {month}', int(year), month_number

In [85]:
lables = pd.DataFrame(periods['period_lbl'])

In [86]:
lables

,period_lbl
0,2MATs: Mar 2023
1,2MATs: Jun 2023
2,2MATs: Sep 2023
3,2MATs: Dec 2023
4,2MATs: Mar 2024
...,...
115,MAT: Aug 2024
116,MAT: Sep 2024
117,Month: Jul 2024
118,Month: Aug 2024


In [62]:
lables[['num','year','month']] = [*lables['period_lbl'].apply(lambda row: get_num_from_period(row))]  

In [69]:
set(lables['period_lbl'][50:60])  - set(periods['period_lbl']) == set()

True

In [98]:
a = set(lables['period_lbl'][50:60])

In [99]:
periods[periods['period_lbl'].isin(a)]

,period_code,period_lbl,label_num
50,51,MAT: Feb 2023,MAT: 2023 (02) Feb
51,52,MAT: Mar 2023,MAT: 2023 (03) Mar
52,53,MAT: Apr 2023,MAT: 2023 (04) Apr
53,54,MAT: May 2023,MAT: 2023 (05) May
54,55,MAT: Jun 2023,MAT: 2023 (06) Jun
55,56,MAT: Jul 2023,MAT: 2023 (07) Jul
56,57,MAT: Aug 2023,MAT: 2023 (08) Aug
57,58,MAT: Sep 2023,MAT: 2023 (09) Sep
58,59,MAT: Oct 2023,MAT: 2023 (10) Oct
59,60,MAT: Nov 2023,MAT: 2023 (11) Nov


In [172]:
periods = pd.read_excel(path, sheet_name ='period_lbl')

In [21]:
new_periods = periods.iloc[20:30,1:]
new_periods['хуй'] = ["хуй"]*len(new_periods)
new_periods['period_batch'] = periods['period_batch'].max()+1

In [22]:
new_periods

,period_lbl,label_num,period_batch,хуй
20,3MMT: Sep 2022,3MMT: 2022 (09) Sep,1,хуй
21,3MMT: Oct 2022,3MMT: 2022 (10) Oct,1,хуй
22,3MMT: Nov 2022,3MMT: 2022 (11) Nov,1,хуй
23,3MMT: Dec 2022,3MMT: 2022 (12) Dec,1,хуй
24,3MMT: Jan 2023,3MMT: 2023 (01) Jan,1,хуй
25,3MMT: Feb 2023,3MMT: 2023 (02) Feb,1,хуй
26,3MMT: Mar 2023,3MMT: 2023 (03) Mar,1,хуй
27,3MMT: Apr 2023,3MMT: 2023 (04) Apr,1,хуй
28,3MMT: May 2023,3MMT: 2023 (05) May,1,хуй
29,3MMT: Jun 2023,3MMT: 2023 (06) Jun,1,хуй


In [23]:
periods = pd.concat([periods,new_periods[['period_lbl','label_num','period_batch' ]]], ignore_index=True)
periods

,period_code,period_lbl,label_num,period_batch
0,1.0,2MATs: Mar 2023,2MATs: 2023 (03) Mar,0
1,2.0,2MATs: Jun 2023,2MATs: 2023 (06) Jun,0
2,3.0,2MATs: Sep 2023,2MATs: 2023 (09) Sep,0
3,4.0,2MATs: Dec 2023,2MATs: 2023 (12) Dec,0
4,5.0,2MATs: Mar 2024,2MATs: 2024 (03) Mar,0
...,...,...,...,...
105,NaN,3MMT: Feb 2023,3MMT: 2023 (02) Feb,1
106,NaN,3MMT: Mar 2023,3MMT: 2023 (03) Mar,1
107,NaN,3MMT: Apr 2023,3MMT: 2023 (04) Apr,1
108,NaN,3MMT: May 2023,3MMT: 2023 (05) May,1


In [24]:
periods = periods.drop('period_code',axis=1).reset_index().rename({'index':'period_code'},axis=1)

In [25]:
periods['period_code'] = periods['period_code']+1 
periods

,period_code,period_lbl,label_num,period_batch
0,1,2MATs: Mar 2023,2MATs: 2023 (03) Mar,0
1,2,2MATs: Jun 2023,2MATs: 2023 (06) Jun,0
2,3,2MATs: Sep 2023,2MATs: 2023 (09) Sep,0
3,4,2MATs: Dec 2023,2MATs: 2023 (12) Dec,0
4,5,2MATs: Mar 2024,2MATs: 2024 (03) Mar,0
...,...,...,...,...
105,106,3MMT: Feb 2023,3MMT: 2023 (02) Feb,1
106,107,3MMT: Mar 2023,3MMT: 2023 (03) Mar,1
107,108,3MMT: Apr 2023,3MMT: 2023 (04) Apr,1
108,109,3MMT: May 2023,3MMT: 2023 (05) May,1


In [29]:
need_lbl = periods[periods['period_batch'] == periods['period_batch'].max() ]['period_code']

In [42]:
type(need_lbl)

pandas.core.series.Series

In [58]:
f = 1

In [61]:
period_batch = 2

In [62]:
f"{ f'period_batch_{period_batch}' if f  else ''}"

'period_batch_2'

In [40]:
!pip install py7zr

^C


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable
  Using cached py7zr-0.22.0-py3-none-any.whl.metadata (16 kB)
  Using cached texttable-1.7.0-py2.py3-none-any.whl.metadata (9.8 kB)
  Using cached pycryptodomex-3.21.0-cp36-abi3-win_amd64.whl.metadata (3.4 kB)
  Using cached pyzstd-0.16.2-cp312-cp312-win_amd64.whl.metadata (2.5 kB)
  Using cached pyppmd-1.1.0-cp312-cp312-win_amd64.whl.metadata (5.8 kB)
  Using cached pybcj-1.0.2-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
  Using cached multivolumefile-0.2.3-py3-none-any.whl.metadata (6.3 kB)
  Using cached inflate64-1.0.0-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
  Using cached Brotli-1.1.0-cp312-cp312-win_amd64.whl.metadata (5.6 kB)
Using cached py7zr-0.22.0-py3-none-any.whl (67 kB)
Using cached Brotli-1.1.0-cp312-cp312-win_amd64.whl (357 kB)
Using cached inflate64-1.0.0-cp312-cp312-win_amd64.whl (35 kB)
Using cached multivolumefile-0.2.3-py3-none-any.whl (17 kB)
Using cached pybcj-1.0.2-cp312-cp312-win_a

In [41]:
from zipfile import ZipFile
import py7zr

In [35]:
dir_of_new = Path(r'C:\Users\nyzhur\Git\pg-dt-portal-for-nz\data\new_format_data' )

In [55]:
zip_files = list(dir_of_new.glob('*.zip')) + list(dir_of_new.glob('*.7z'))



In [36]:
dfs = []
pattern = '_allshops.parquet'

In [56]:
paths = Path(r'C:\Users\nyzhur\Git\pg-dt-portal-for-nz\data\new_format_data')
zip_files = list(paths.glob('*.zip')) 
tmp_zip = Path('./data/tmp/7z')
blya =list(paths.glob('*.7z'))
tmp_7z = Path('./data/tmp/7z')
pattern = '_allshops.parquet'

In [ ]:
dfs = []
for zip_f in zip_files:
    with ZipFile(zip_f) as zip_file: 
        info = zip_file.namelist() 
        for name in info:
            if name.count(pattern):
                print(name)
                category = name.split('/')[1].replace(pattern,'')
                tmp_df  = pd.read_parquet(zip_file.extract(name, path = tmp_zip )) 
                tmp_df['category'] = [category]*len(tmp_df)
                dfs.append(tmp_df)

In [ ]:
py7zr

In [57]:
dfs = []
for b in blya:
    with py7zr.SevenZipFile(zip_f, 'r') as archive:
        info = archive.namelist() 
        names = [name for name in info if name.count(pattern) ]
        archive.reset()
        #archive.extract(path = tmp_7z, targets=names)
        for name in names:
            tmp_df = pd.read_parquet(tmp_7z.joinpath(name.split('/')[1])) 
            category = name.split('/')[1].replace(pattern,'')
            tmp_df['category'] = [category]*len(tmp_df)
            dfs.append(tmp_df)
            
            #category = name.split('/')[1].replace(pattern,'')
            #tmp_df = pd.read_parquet('./data/tmp/7z'+name)
        # for name in info:
        #     if name.count(pattern):
        #         print(name)
        #         category = name.split('/')[1].replace(pattern,'')
        #         tmp = archive.extract(path = './data/tmp/7z', targets=[name])
        #         tmp_df = pd.read_parquet(tmp)
        #         #tmp_df['category'] = [category]*len(tmp_df)
        #         #dfs.append(tmp_df)

FileNotFoundError: [Errno 2] No such file or directory: 'data\\tmp\\7z\\br_allshops.parquet'

In [12]:
df =dfs[0] 

In [ ]:
def add_period_lbls(df_in,periods):

    time_period_type_dict = {24 :'2MATs/ 104 we', 12:'MAT/ 52 we', 3:'3MMT/ 12we', 1: 'Month'}
    
    col_to_int = ['duration', 'year', 'month']
    df_in[col_to_int] = df_in[col_to_int].astype(int)
    
    df_in['time_period_type'] = df_in['duration'].map(time_period_type_dict)
    
    df_tmp_per = df_in[['time_period_type', 'year', 'month']].drop_duplicates()
    df_tmp_per['period_lbl'] = df_tmp_per[['time_period_type', 'year', 'month']]\
        .apply(lambda row: get_period_lbl(*row), axis=1)
    
    df_tmp_per['label_num'] = df_tmp_per[['time_period_type', 'year', 'month']]\
        .apply(lambda row: get_label_num(*row), axis=1)


    new_periods = set(df_tmp_per['period_lbl']) - set(periods['period_lbl'])
    if len(new_periods):
        print('New periods!')
        print(*new_periods)
        new_periods_df = df_tmp_per[df_tmp_per['period_lbl'].isin(new_periods)].sort_values(by=['year','month'])
        new_periods_df['period_batch'] = periods['period_batch'].max()+1
        periods = pd.concat([periods, new_periods_df[['period_lbl','label_num','period_batch' ]]],ignore_index=True )
        periods['period_code'] = range(len(periods))+ 1
        with pd.ExcelWriter("my_sheet.xlsx",engine="openpyxl",mode="a",on_sheet_exists="replace") as writer:
            periods.to_excel(writer, sheet_name ='period_lbl', if_sheet_exists='replace')
            
        print('New periods added! Period batch', periods['period_batch'].max())

In [21]:
def get_period_lbl(time_period_type, year, month):
    return f"{time_period_type.split('/')[0]}: {calendar.month_abbr[month]} {year}"

def get_label_num(time_period_type, year, month):
    return f"{time_period_type.split('/')[0]}: {year} ({str(month).zfill(2)}) {calendar.month_abbr[month]}"


In [24]:
add_period_lbls(df, periods)

New periods!
2MATs: Sep 2024 MAT: Sep 2024 3MMT: Sep 2024


AttributeError: module 'pandas' has no attribute 'to_excel'

In [6]:
periods

,Unnamed: 0.1,period_code,Unnamed: 0,period_lbl,label_num,period_batch
0,0,1,0.0,2MATs: Mar 2023,2MATs: 2023 (03) Mar,0
1,1,2,1.0,2MATs: Jun 2023,2MATs: 2023 (06) Jun,0
2,2,3,2.0,2MATs: Sep 2023,2MATs: 2023 (09) Sep,0
3,3,4,3.0,2MATs: Dec 2023,2MATs: 2023 (12) Dec,0
4,4,5,4.0,2MATs: Mar 2024,2MATs: 2024 (03) Mar,0
...,...,...,...,...,...,...
105,105,106,NaN,3MMT: Aug 2024,3MMT: 2024 (08) Aug,1
106,106,107,NaN,MAT: Sep 2024,MAT: 2024 (09) Sep,1
107,107,108,NaN,2MATs: Sep 2024,2MATs: 2024 (09) Sep,1
108,108,109,NaN,3MMT: Sep 2024,3MMT: 2024 (09) Sep,1


In [15]:
def drop_column():
    global periods
    periods =periods.drop('Unnamed: 0.1', axis=1)

In [16]:
drop_column()

In [17]:
periods

,period_code,Unnamed: 0,period_lbl,label_num,period_batch
0,1,0.0,2MATs: Mar 2023,2MATs: 2023 (03) Mar,0
1,2,1.0,2MATs: Jun 2023,2MATs: 2023 (06) Jun,0
2,3,2.0,2MATs: Sep 2023,2MATs: 2023 (09) Sep,0
3,4,3.0,2MATs: Dec 2023,2MATs: 2023 (12) Dec,0
4,5,4.0,2MATs: Mar 2024,2MATs: 2024 (03) Mar,0
...,...,...,...,...,...
105,106,NaN,3MMT: Aug 2024,3MMT: 2024 (08) Aug,1
106,107,NaN,MAT: Sep 2024,MAT: 2024 (09) Sep,1
107,108,NaN,2MATs: Sep 2024,2MATs: 2024 (09) Sep,1
108,109,NaN,3MMT: Sep 2024,3MMT: 2024 (09) Sep,1


In [7]:
res = pd.read_parquet('data/new_data_merged/new_data.pq')

In [65]:
chk_na = res.isna().sum()
if chk_na[chk_na > 0].shape[0] > 0:
    print('WARNING\n', 'columns with na:')
    print(chk_na[chk_na > 0])

WARNING
 columns with na:
Unnamed: 0_x    848376
dtype: int64


In [66]:
chk_na[chk_na > 0]

Unnamed: 0_x    848376
dtype: int64

In [71]:
res[res['Unnamed: 0_x'].isnull()]['year'].value_counts()

KeyError: 'year'

In [5]:
example = pd.read_excel('.\codes\Example.xlsx', sheet_name = 'example')

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\nyzhur\AppData\Local\Temp\ipykernel_6136\3172424420.py:1: SyntaxWarning: invalid escape sequence '\c'
  example = pd.read_excel('.\codes\Example.xlsx', sheet_name = 'example')


In [4]:
fin_cols = example.columns.to_list() + ['shop_code', 'shop_lvls']

In [10]:
set(res.columns) -set(fin_cols)

set()

In [11]:
res.head()

,product_lvls,category,demo_lvls,socdem_gr,year,buying_households,raw_shoppers,percent_household_penetration,purchase_frequency,occasions,...,product_hier,demo_hier,products,features,cat_segment,demo_groups,shop_code,shop_lvls,period_lbl,time_period_type
0,0,2,0,1,2024,17439.302619,4386.0,31.963166,1.848940,32244.217603,...,2000,1,2000,0,200,1,1,1,117,2
1,0,2,0,1,2024,11227.172911,2797.0,20.577428,1.637028,18379.196502,...,2000,1,2000,0,200,1,2,2,117,2
2,0,2,0,1,2024,3474.668642,901.0,6.368455,1.451968,5045.106231,...,2000,1,2000,0,200,1,3,2,117,2
3,0,2,0,1,2024,2633.203144,707.0,4.826197,1.439084,3789.399613,...,2000,1,2000,0,200,1,4,2,117,2
4,0,2,0,1,2024,493.310332,154.0,0.904151,1.448256,714.439825,...,2000,1,2000,0,200,1,5,3,117,2


In [22]:
res.query('category == 2')

,product_lvls,category,demo_lvls,socdem_gr,year,buying_households,raw_shoppers,percent_household_penetration,purchase_frequency,occasions,...,product_hier,demo_hier,products,features,cat_segment,demo_groups,shop_code,shop_lvls,period_lbl,time_period_type
0,0,2,0,1,2024,17439.302619,4386.0,31.963166,1.848940,32244.217603,...,2000,1,2000,0,200,1,1,1,117,2
1,0,2,0,1,2024,11227.172911,2797.0,20.577428,1.637028,18379.196502,...,2000,1,2000,0,200,1,2,2,117,2
2,0,2,0,1,2024,3474.668642,901.0,6.368455,1.451968,5045.106231,...,2000,1,2000,0,200,1,3,2,117,2
3,0,2,0,1,2024,2633.203144,707.0,4.826197,1.439084,3789.399613,...,2000,1,2000,0,200,1,4,2,117,2
4,0,2,0,1,2024,493.310332,154.0,0.904151,1.448256,714.439825,...,2000,1,2000,0,200,1,5,3,117,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632214,1,2,0,1,2024,10.268068,2.0,0.018820,1.000532,10.273533,...,2004,1,2004,1,205,1,8,3,109,1
632215,2,2,0,1,2024,23.158147,7.0,0.042445,1.000501,23.169754,...,2009,1,2009,1,201,1,20,4,109,1
632216,2,2,0,1,2024,21.520727,8.0,0.039444,1.000474,21.530936,...,2007,1,2007,7,202,1,22,4,109,1
632217,5,2,0,1,2024,53.203362,6.0,0.097512,1.001536,53.285088,...,2012,1,2012,17,205,1,19,3,109,1


In [19]:
res[res['period_lbl'].isin([101,102,103])]

,product_lvls,category,demo_lvls,socdem_gr,year,buying_households,raw_shoppers,percent_household_penetration,purchase_frequency,occasions,...,product_hier,demo_hier,products,features,cat_segment,demo_groups,shop_code,shop_lvls,period_lbl,time_period_type
1068207,1,6,0,1,2024,2451.536642,695.0,4.493234,1.502073,3682.386699,...,6084,1,6084,1,602,1,29,2,101,2
1068208,3,6,0,1,2024,140.391420,20.0,0.257313,1.069643,150.168743,...,6149,1,6149,14,600,1,47,2,101,2
1068209,2,6,0,1,2024,1192.840777,373.0,2.186267,1.368193,1632.036008,...,6085,1,6085,2,602,1,30,2,101,2
1068210,1,6,0,1,2024,141.944324,62.0,0.260159,1.718480,243.928497,...,6128,1,6128,1,603,1,29,2,101,2
1068211,4,6,0,1,2024,10.548472,7.0,0.019333,1.199482,12.652699,...,6060,1,6060,15,600,1,9,3,101,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5032571,2,4,0,1,2024,61.396827,26.0,0.112530,1.001095,61.464083,...,4177,1,4177,2,400,1,32,3,103,1
5032572,1,4,0,1,2024,3.939327,2.0,0.007220,1.000379,3.940822,...,4211,1,4211,1,400,1,43,2,103,1
5032573,1,4,0,1,2024,21.024898,9.0,0.038535,1.000305,21.031313,...,4182,1,4182,1,400,1,30,2,103,1
5032574,1,4,0,1,2024,6.647788,2.0,0.012184,1.000417,6.650562,...,4195,1,4195,1,400,1,5,3,103,1


In [16]:
try:
    len(df)
except NameError:
    df = pd.DataFrame()
    print('He fortanulo') 

He fortanulo


In [1]:
a =b'asa'

In [2]:
print(a)

b'asa'
